# Bag of Words

Classificação de texto é uma das tarefas mais comuns em que processamento de Língua Natural pode atuar.

Algoritmos de classificação normalmente precisam de uma representação numérica servindo de entrada para poder gerar uma saída (classificação). O processamento do texto então entra para, de alguma maneira, converter um texto para uma representação numérica.

Uma maneira simples e que muitas vezes já é suficiente para realizar essa **transformação** de um texto para números é contar a quantidade de vezes que uma palavra aparece em um texto. Essa técnica de transformação é chamada de Bag of Words.



## Exemplo

Para mostrar como fica a representação numérica de textos com Bag of Words (BoW) usaremos o objeto *CountVectorizer* do sklearn (que faz esse processo de transformação).

Porém, primeiro é necessário ter os textos. Aqui criamos uma possível representação usando *numpy*.

In [16]:
import numpy as np

texto = np.array(["Você", 
                  "Gostava tanto de você", 
                  "Pede a ela", 
                  "Ela partiu", 
                  "Você e eu, eu e você"])
texto

array(['Você', 'Gostava tanto de você', 'Pede a ela', 'Ela partiu',
       'Você e eu, eu e você'], dtype='<U21')

Passamos esse texto para ser reconhecido e transformado pelo CountVectorizer. O *sklearn* representa a saída numérica dos textos como uma matriz esparsa, onde as colunas são palavras e as linhas são os textos. Cada entrada dessa matriz terá a quantidade de vezes que uma palavra aparece nesse texto e, para evitar alocar muitos espaços com zero, o *sklearn* compacta essa saída em uma matriz esparsa.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()

X = bow.fit_transform(texto)
X

<5x8 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

Uma matriz esparsa pode ser vista como uma matriz "normal" usando o método *toarray()*, exemplificado na sequência. Para identificar as colunas usadas, basta usar *get_feature_names_out* no Bag of Words treinado.

In [18]:
import pandas as pd

pd.DataFrame(X.toarray(), 
             columns = bow.get_feature_names_out(),
             index = texto)

,de,ela,eu,gostava,partiu,pede,tanto,você
Você,0,0,0,0,0,0,0,1
Gostava tanto de você,1,0,0,1,0,0,1,1
Pede a ela,0,1,0,0,0,1,0,0
Ela partiu,0,1,0,0,1,0,0,0
"Você e eu, eu e você",0,0,2,0,0,0,0,2


Agora, ao invés de passar os textos como entrada do classificador, passamos a representação numérica.

# Exemplo de Classificação

Aqui temos como exemplo um conjunto de dados em que uma coluna representa o texto de um SMS e outra coluna se ele foi classificado como spam ou não (classe ham). As linhas a seguir fazem a leitura do dataset e uma limpeza / organização dos dados para termos apenas o necessário.

In [19]:
# opcional - caminho do csv está no seu google drive
# from google.colab import drive
# drive.mount("/content/drive/")

In [20]:
import pandas as pd

sms = pd.read_csv('https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv', encoding='latin1')
sms = sms.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
sms.columns = ['classe', 'texto']
sms.head()

,classe,texto
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Como é um problema da classificação, precisamos determinar nosso *target* (y) e a entrada (X).

In [21]:
X = sms['texto']
y = sms['classe']

Para este exemplo de introdução ao tema, apenas separamos aleatoriamente os dados entre treino e teste.

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.2)

Então, criamos nosso bag of words com os dados de treino, e com ele fazemos a transformação dos textos de treino e teste para uma representação numérica.

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer, word_tokenize

# bow = CountVectorizer(tokenizer=TweetTokenizer().tokenize)
# bow = CountVectorizer(tokenizer=word_tokenize)
bow = CountVectorizer()
X_train = bow.fit_transform(X_train)
X_test = bow.transform(X_test)

Assim, podemos usar o X (a matriz esparsa da frequência das palavras) como entrada do algoritmo.

In [25]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

Enfim, podemos identificar o êxito na classificação.

Observação: Não aplicamos nenhuma técnica de rebalanceamento das classes neste notebook, mas uma redução aleatória dos dados da classe majoritária (Random Under Sampling) levava a uma acurácia próxima de 0.9.

In [26]:
from sklearn.metrics import classification_report

print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

         ham       1.00      0.98      0.99       974
        spam       0.90      0.99      0.95       141

    accuracy                           0.99      1115
   macro avg       0.95      0.99      0.97      1115
weighted avg       0.99      0.99      0.99      1115



Bag of Words é uma técnica usada em Processamento de Língua Natural que auxilia na classificação de texto, por representar numericamente valores que antes eram caracteres de uma string. 

Para alguns problemas essa abordagem é realmente muito boa. Em outros casos porém, por seguir uma abordagem *naive* para transformar textos em números, pode ser insuficiente para gerar bons resultados e sendo necessário utilizar outra abordagem ou voltar a analisar os dados e identificar se o problema que esteja lidando realmente é viável para envolver aprendizado de máquina e classificação como um todo.